## Read Data From SQL Pool


In [1]:
val df_City = spark.read.sqlanalytics("accsynapsepools.wwi.DimCity") 
val df_FactOrder = spark.read.sqlanalytics("accsynapsepools.wwi.FactOrder")
 // df.show(10)

df_City: org.apache.spark.sql.DataFrame = [CityKey: int, WWICityID: int ... 11 more fields]
df_FactOrder: org.apache.spark.sql.DataFrame = [OrderKey: bigint, CityKey: int ... 17 more fields]

In [2]:
df_City.createOrReplaceTempView("city")
df_FactOrder.createOrReplaceTempView("factOrder")

## Data Exploration and Analysis


In [3]:
val df = spark.sql("select * from factOrder")

df: org.apache.spark.sql.DataFrame = [OrderKey: bigint, CityKey: int ... 17 more fields]

## Use of Magic commands


In [4]:
%%sql 
SELECT * FROM city

## Join Data


In [5]:
val df = (spark.sql("""SELECT city.StateProvince, SUM(Quantity) as TotalOrders 
                    FROM factorder 
                    JOIN  city on city.CityKey=factorder.CityKey 
                    group by city.StateProvince 
                    order by SUM(Quantity) desc """))
display(df.limit(10))

## Write result to ADLS Gen2 in Parquet format


In [7]:
// Primary storage info
val account_name = "accsynapsestorage" // fill in your primary account name
val container_name = "curateddata" // fill in your container name
val relative_path = "ordersumamtion" // fill in your relative folder path

val adls_path = f"abfss://$container_name@$account_name.dfs.core.windows.net/$relative_path/"
print("Primary storage account path: " + adls_path)
val parquet_path = adls_path + "OrdersByState"
df.write.mode("Overwrite").parquet(parquet_path)


account_name: String = accsynapsestorage
container_name: String = curateddata
relative_path: String = ordersumamtion
adls_path: String = abfss://curateddata@accsynapsestorage.dfs.core.windows.net/ordersumamtion/
Primary storage account path: abfss://curateddata@accsynapsestorage.dfs.core.windows.net/ordersumamtion/parquet_path: String = abfss://curateddata@accsynapsestorage.dfs.core.windows.net/ordersumamtion/OrdersByState